# NLP - Q3a 
# (Webscraping ESPNCricinfo page & downloading tabular data to Database File)
## Batch 23 Term 4
## Group 12 Assignment
### By:
####    Bharatvamsi Kanchi    - 12420076
####    Phani Ramachandra     - 12420063
####    Swastika Mishra       - 12420008
####    V Hemanth Kumar       - 12420085

In [2]:
# Install necessary libraries
!pip install selenium pandas beautifulsoup4 webdriver-manager --quiet

In [3]:
# Importing all the required libraries

import numpy as np
import pandas as pd
import time
import re
from io import StringIO
from IPython.display import Markdown

from bs4 import BeautifulSoup


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

## Scraping web page content from ESPNCricinfo website

In [5]:
# Set-up Chrome browwser and launch URL using chrome driver

# Initiate Driver
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

# Load URL page

url = "https://www.espncricinfo.com/series/icc-champions-trophy-2024-25-1459031/india-vs-new-zealand-final-1466428/full-scorecard"
driver.get(url)
time.sleep(8)  # Delay to ensure contents loaded on chrome browser
print(f"✅ Page Title: {driver.title}")

# Parse the html page with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

✅ Page Title: IND vs NZ Cricket Scorecard, Final at Dubai, March 09, 2025


In [6]:
# Configure a SQL lite server database
import sqlite3

db = 'scorecard.db'

In [7]:
# Defining a Function to extract required tabular information from parsed page at soup 
# Extract tabular data and push to SQL database

def extract(soup, db_name):
    
    all_dataframes = {}
    
    try:
        # Using basic pandas to read all tables from the raw HTML content
        tables = pd.read_html(StringIO(str(soup)))
    except ValueError:
        print("❌ Could not find any tables on the page using pandas.read_html.")
        return all_dataframes
    
    if len(tables) < 4:
        print(f"Found only {len(tables)} tables. Expected at least 4 for a full match scorecard.")
        return all_dataframes

    def clean_batting_df(df, name):
        df_clean = df.iloc[:, :7]   
        df_clean.columns = ['Batsman', 'Dismissal', 'R', 'B', '4s', '6s', 'SR']
        
        #  Filter out rows where the 'Batsman' name is NaN (removes blank rows)
        df_clean = df_clean.dropna(subset=['Batsman'])
        
        #  Filter out rows that are 'Extras', 'Total', or 'Did not bat'
        df_clean = df_clean[~df_clean['Batsman'].str.contains('Extras|TOTAL|Did not bat|Fall of wickets', case=False, na=False)]
        
        # Clean up player names (remove symbols like †, ‡)
        df_clean['Batsman'] = df_clean['Batsman'].astype(str).str.replace(r'[†‡(].*', '', regex=True).str.strip()
        
        all_dataframes[name] = df_clean
        print(f"Extracted and cleaned: {name}")

    def clean_bowling_df(df, name):
        df_clean = df.iloc[:, :10].copy()
        df_clean.columns = ['Bowler', 'O', 'M', 'R', 'W', 'Econ', '0s', '4s', '6s', 'WD']
        
        # 1. Filter out rows where 'O' (Overs) is not a number (removes commentary)
        df_clean['O_numeric'] = pd.to_numeric(df_clean['O'], errors='coerce')
        df_clean = df_clean.dropna(subset=['O_numeric'])
        df_clean = df_clean.drop(columns=['O_numeric'])
        
        all_dataframes[name] = df_clean
        print(f"Extracted and cleaned: {name}")

    # --- Innings Extraction ---
    clean_batting_df(tables[0], 'innings1_batting')
    clean_bowling_df(tables[1], 'innings1_bowling')
    clean_batting_df(tables[2], 'innings2_batting')
    clean_bowling_df(tables[3], 'innings2_bowling')
    
    # --- Store to SQLite ---
    if all_dataframes:
        print(f"\n Storing {len(all_dataframes)} tables to {db_name}...")
        conn = sqlite3.connect(db_name)
        for table_name, df in all_dataframes.items():
            # Store the DataFrame directly to a new table in SQLite
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            print(f"  -> Table '{table_name}' saved.")
        conn.close()
        print(" ESPN Cricinfo Scores data saved successfully.")
    
    return all_dataframes



In [8]:
# Process and save the data in database
all_dataframes = extract(soup, db)

Extracted and cleaned: innings1_batting
Extracted and cleaned: innings1_bowling
Extracted and cleaned: innings2_batting
Extracted and cleaned: innings2_bowling

 Storing 4 tables to scorecard.db...
  -> Table 'innings1_batting' saved.
  -> Table 'innings1_bowling' saved.
  -> Table 'innings2_batting' saved.
  -> Table 'innings2_bowling' saved.
 ESPN Cricinfo Scores data saved successfully.


In [9]:
# Connect to the database
conn = sqlite3.connect(db)

# Load each table into a DataFrame
NZ_batting = pd.read_sql_query("SELECT * FROM innings1_batting", conn)
NZ_bowling = pd.read_sql_query("SELECT * FROM innings1_bowling", conn)
India_batting = pd.read_sql_query("SELECT * FROM innings2_batting", conn)
India_bowling = pd.read_sql_query("SELECT * FROM innings2_bowling", conn)

# Close connection
conn.close()

# Display them
print("New Zealand Batting")
display(NZ_batting)

print("\n New Zealand Bowling")
display(NZ_bowling)

print("\n India Batting")
display(India_batting)

print("\n India Bowling")
display(India_bowling)

New Zealand Batting


,Batsman,Dismissal,R,B,4s,6s,SR
0,Will Young,lbw b Varun,15,23,41,2,0
1,Rachin Ravindra,b Kuldeep Yadav,37,29,52,4,1
2,Kane Williamson,c & b Kuldeep Yadav,11,14,19,1,0
3,Daryl Mitchell,c Sharma b Mohammed Shami,63,101,132,3,0
4,Tom Latham,lbw b Jadeja,14,30,40,0,0
5,Glenn Phillips,b Varun,34,52,46,2,1
6,Michael Bracewell,not out,53,40,57,3,2
7,Mitchell Santner,run out (Kohli/†Rahul),8,10,17,0,0
8,Nathan Smith,not out,0,1,4,0,0



 New Zealand Bowling


,Bowler,O,M,R,W,Econ,0s,4s,6s,WD
0,Mohammed Shami,9,0,74,1,8.22,21,9,1,1
1,Hardik Pandya,3,0,30,0,10.00,8,2,2,3
2,Varun Chakravarthy,10,0,45,2,4.50,31,0,0,3
3,Kuldeep Yadav,10,0,40,2,4.00,34,2,1,1
4,Axar Patel,8,0,29,0,3.62,25,1,0,1
5,Ravindra Jadeja,10,0,30,1,3.00,35,1,0,0



 India Batting


,Batsman,Dismissal,R,B,4s,6s,SR
0,Rohit Sharma,st †Latham b Ravindra,76,83,102,7,3
1,Shubman Gill,c Phillips b Santner,31,50,75,0,1
2,Virat Kohli,lbw b Bracewell,1,2,4,0,0
3,Shreyas Iyer,c Ravindra b Santner,48,62,68,2,2
4,Axar Patel,c O'Rourke b Bracewell,29,40,58,1,1
5,KL Rahul,not out,34,33,47,1,1
6,Hardik Pandya,c & b Jamieson,18,18,27,1,1
7,Ravindra Jadeja,not out,9,6,7,1,0



 India Bowling


,Bowler,O,M,R,W,Econ,0s,4s,6s,WD
0,Kyle Jamieson,5,0,24,1,4.80,19,1,1,1
1,Will O’Rourke,7,0,56,0,8.00,15,6,0,3
2,Nathan Smith,2,0,22,0,11.00,6,2,2,0
3,Mitchell Santner,10,0,46,2,4.60,34,2,2,1
4,Rachin Ravindra,10,1,47,1,4.70,30,1,2,1
5,Michael Bracewell,10,1,28,2,2.80,39,0,1,0
6,Glenn Phillips,5,0,31,0,6.20,8,1,1,0
